In [1]:
library("tidyverse")
library("stringr")

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


In [2]:
guess_encoding("../fuente/cvu/PERSONA_FISICA/PERSONA_FISICA.csv", n_max = -1)

encoding,confidence
ISO-8859-1,0.23


In [3]:
cvu <- read_csv("../fuente/cvu/PERSONA_FISICA/PERSONA_FISICA.csv", locale = locale(encoding = "ISO-8859-1"))%>% 
rename(nombres = NOMBRE, 
       primer_apellido = APELLIDO_PATERNO,
       segundo_apellido = APELLIDO_MATERNO) %>% 
select(-ID_PERSONA_FISICA, 
       -ID_ESTATUS_MIGRATORIO, 
       -ID_ESTADO_CIVIL, 
       -ID_NACIONALIDAD, 
       -ID_ENTIDAD_NACIMIENTO, 
       -IND_DATO_CORRECTO, 
       -IND_TIPO_BENEFICIO, 
       -USUARIO_ALTA, 
       -USUARIO_MODIFICACION, 
       -FECHA_ALTA, 
       -FECHA_MODIFICACION, 
       -IND_ESTATUS)

head(cvu)
dim(cvu)

Parsed with column specification:
cols(
  .default = col_integer(),
  ID_ESTATUS_MIGRATORIO = col_character(),
  NOMBRE = col_character(),
  APELLIDO_PATERNO = col_character(),
  APELLIDO_MATERNO = col_character(),
  FECHA_NACIMIENTO = col_character(),
  CURP = col_character(),
  NUMERO_CVU = col_character(),
  FECHA_ALTA = col_character(),
  FECHA_MODIFICACION = col_character()
)
See spec(...) for full column specifications.


ID_GENERO,ID_PAIS_NACIMIENTO,nombres,primer_apellido,segundo_apellido,FECHA_NACIMIENTO,CURP,NUMERO_CVU
1,NA,MANUEL ALFREDO,HERNANDEZ,OCHOA,30/09/61,AAPF610930HJCLDR01,NA
1,143,JESUS,MONREAL,GARCIA,26/08/16,MOGJ800924HDFNRS00,779775
1,143,IGNACIO,MORENO,VAZQUEZ,31/07/61,MOVI610731HDFRZG05,344041
1,143,ROBERTO ARTURO,COLIN,RUIZ,20/09/76,CORR760920HDFLZB07,227332
1,143,GUSTAVO ADAD,RIOS,ZUÑIGA,09/02/88,RIZG880209HDFSXS00,378676
2,143,ITZEL NANDHY,MORENO,CARRILLO,28/12/85,MOCI851228MDFRRT05,NA


[1] 284177      8

In [4]:
cat_genero <- read_csv("../fuente//cvu/PERSONA_FISICA/CAT_GENERO.csv") %>% 
select(ID_GENERO, DESC_GENERO)

head(cat_genero)

Parsed with column specification:
cols(
  ID_GENERO = col_integer(),
  CVE_GENERO = col_character(),
  DESC_GENERO = col_character(),
  USUARIO_ALTA = col_integer(),
  USUARIO_MODIFICACION = col_character(),
  FECHA_ALTA = col_character(),
  FECHA_MODIFICACION = col_character(),
  IND_ESTATUS = col_integer()
)


ID_GENERO,DESC_GENERO
1,Masculino
2,Femenino


In [5]:
cvu_con_genero <- left_join(cvu, cat_genero) %>% 
select(-ID_GENERO) %>% 
rename(genero = DESC_GENERO)

head(cvu_con_genero)
dim(cvu_con_genero)

Joining, by = "ID_GENERO"


ID_PAIS_NACIMIENTO,nombres,primer_apellido,segundo_apellido,FECHA_NACIMIENTO,CURP,NUMERO_CVU,genero
NA,MANUEL ALFREDO,HERNANDEZ,OCHOA,30/09/61,AAPF610930HJCLDR01,NA,Masculino
143,JESUS,MONREAL,GARCIA,26/08/16,MOGJ800924HDFNRS00,779775,Masculino
143,IGNACIO,MORENO,VAZQUEZ,31/07/61,MOVI610731HDFRZG05,344041,Masculino
143,ROBERTO ARTURO,COLIN,RUIZ,20/09/76,CORR760920HDFLZB07,227332,Masculino
143,GUSTAVO ADAD,RIOS,ZUÑIGA,09/02/88,RIZG880209HDFSXS00,378676,Masculino
143,ITZEL NANDHY,MORENO,CARRILLO,28/12/85,MOCI851228MDFRRT05,NA,Femenino


[1] 284177      8

In [6]:
cat_pais <- read_csv("../fuente/cvu/PERSONA_FISICA/CAT_PAIS.csv") %>% 
select(ID_PAIS, CVE_PAIS)

head(cat_pais)

Parsed with column specification:
cols(
  ID_PAIS = col_integer(),
  CVE_PAIS = col_character(),
  DESC_PAIS = col_character(),
  USUARIO_ALTA = col_integer(),
  USUARIO_MODIFICACION = col_character(),
  FECHA_ALTA = col_character(),
  FECHA_MODIFICACION = col_character(),
  IND_ESTATUS = col_integer()
)


ID_PAIS,CVE_PAIS
119,KOR
120,KWT
121,KGZ
122,LAO
123,LVA
124,LBN


In [7]:
cvu_con_pais <- left_join(cvu_con_genero, cat_pais, by = c("ID_PAIS_NACIMIENTO" = "ID_PAIS")) %>% 
select(-ID_PAIS_NACIMIENTO) %>% 
rename(pais_de_nacimiento = CVE_PAIS, 
      curp = CURP,
      cvu = NUMERO_CVU) %>% 
mutate(dni = NA,
      afiliacion = NA,
      orcid = NA,
      rn = NA,
      pais_asociado = NA,
      origen = "cvu") %>% 
mutate(fecha_de_nacimiento = str_replace(FECHA_NACIMIENTO,
                                         pattern = "/(\\d\\d)$",
                                         replacement = "/19\\1")) %>% 
select(-FECHA_NACIMIENTO)

head(cvu_con_pais)
dim(cvu_con_pais)

nombres,primer_apellido,segundo_apellido,curp,cvu,genero,pais_de_nacimiento,dni,afiliacion,orcid,rn,pais_asociado,origen,fecha_de_nacimiento
MANUEL ALFREDO,HERNANDEZ,OCHOA,AAPF610930HJCLDR01,NA,Masculino,NA,NA,NA,NA,NA,NA,cvu,30/09/1961
JESUS,MONREAL,GARCIA,MOGJ800924HDFNRS00,779775,Masculino,MEX,NA,NA,NA,NA,NA,cvu,26/08/1916
IGNACIO,MORENO,VAZQUEZ,MOVI610731HDFRZG05,344041,Masculino,MEX,NA,NA,NA,NA,NA,cvu,31/07/1961
ROBERTO ARTURO,COLIN,RUIZ,CORR760920HDFLZB07,227332,Masculino,MEX,NA,NA,NA,NA,NA,cvu,20/09/1976
GUSTAVO ADAD,RIOS,ZUÑIGA,RIZG880209HDFSXS00,378676,Masculino,MEX,NA,NA,NA,NA,NA,cvu,09/02/1988
ITZEL NANDHY,MORENO,CARRILLO,MOCI851228MDFRRT05,NA,Femenino,MEX,NA,NA,NA,NA,NA,cvu,28/12/1985


[1] 284177     14

In [9]:
write_csv(cvu_con_pais, "../datos/cvu.csv")